In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import copy
import os
import numpy as np
import torch
import torch.distributions as dist
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
import matplotlib
import seaborn as sns
from tqdm.auto import tqdm
from signatureshape.animation import fetch_animations
import extratorch as etorch
import shapeflow as sf

In [ ]:
# make reproducible
seed = torch.manual_seed(0)
# better plotting

set_matplotlib_formats("pdf", "svg")
matplotlib.rcParams.update({"font.size": 12})
set_matplotlib_formats("pdf", "svg")
plt.style.use("tableau-colorblind10")
sns.set_style("white")

In [ ]:
# fetch data as so3
# we assume all have the same skeleton
print("Loading mocap data:")
# walk  data
walk_subjects = ["07", "08", "35", "16"]

walk_animations = []
walk_desc = []
for s in walk_subjects:
    for t in fetch_animations(100, subject_file_name=(s + ".asf")):
        if t[2][:4] == "walk":
            walk_animations.append(t[1])
            walk_desc.append(t[2])

walk_animations_train_frame = sum(
    len(anim.get_frames()) for anim in walk_animations[:18]
)

# run data
run_subjects = ["09", "16", "35"]
run_animations = []
run_skeletons = []
for s in run_subjects:

    for t in fetch_animations(100, subject_file_name=(s + ".asf")):
        if t[2][:3] == "run":
            run_skeletons.append(t[0])
            run_animations.append(t[1])

print("Convert to array:")
walk_angle_array = sf.utils.animation_to_eulers(
    walk_animations,
    reduce_shape=True,
    remove_root=True,
    deg2rad=True,
    skeleton=run_skeletons[0],
    max_frame_count=240,
)
run_angle_array = sf.utils.animation_to_eulers(
    run_animations,
    reduce_shape=True,
    remove_root=True,
    deg2rad=True,
    skeleton=run_skeletons[0],
    max_frame_count=240,
)

In [ ]:
run_angle_array.shape, walk_angle_array.shape

In [ ]:
walk_angle_tensor_ = torch.tensor(walk_angle_array, dtype=torch.float32)
run_angle_tensor_ = torch.tensor(run_angle_array, dtype=torch.float32)
wr_angle_tensor_ = torch.cat((walk_angle_tensor_, run_angle_tensor_))

# normalize
std, mean = torch.std_mean(wr_angle_tensor_, dim=0)
wr_angle_tensor_norm = (wr_angle_tensor_ - mean) / std
run_angle_tensor_norm = (run_angle_tensor_ - mean) / std
walk_angle_tensor_norm = (walk_angle_tensor_ - mean) / std

nonzero = torch.argwhere(
    torch.sum(torch.abs(torch.diff(wr_angle_tensor_, dim=0)), dim=0) > 0.0
).flatten()
choosen = nonzero[
    [
        0,
        1,
        14,
        26,
        33,
    ]
]
# [[24,26,33,34]]

walk_angle_tensor = walk_angle_tensor_norm[:, choosen]
run_angle_tensor = run_angle_tensor_norm[:, choosen]
wr_angle_tensor = wr_angle_tensor_norm[:, choosen]

In [ ]:
# make priors
run_len = run_angle_tensor.shape[0]
walk_len = walk_angle_tensor.shape[0]
prior_run = torch.cat((torch.ones(run_len), torch.zeros(walk_len)))
# p run, walk
priors = torch.stack((prior_run, abs(prior_run - 1)), dim=1)
eps = torch.rand(len(priors)) * 0.1
priors[:, 1] = 0.5 + eps
priors[:, 0] = 0.5 - eps

In [ ]:
data_walk = torch.utils.data.TensorDataset(walk_angle_tensor)
data_run = torch.utils.data.TensorDataset(run_angle_tensor)
data = torch.utils.data.TensorDataset(wr_angle_tensor, priors, priors)

In [ ]:
#######
DIR = "../figures/cluster_frames/"
SET_NAME = "cont_euler_4"
PATH_FIGURES = os.path.join(DIR, SET_NAME)
if not os.path.exists(PATH_FIGURES):
    os.makedirs(PATH_FIGURES)
########

event_shape = data[0][0].shape
base_dist = dist.Independent(
    dist.Normal(loc=torch.zeros(event_shape), scale=torch.ones(event_shape)), 1
)

lr_scheduler = lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(
    optim, mode="min", factor=0.5, patience=5, verbose=True
)
MODEL_PARAMS = {
    "model": sf.nf.get_flow,
    "get_transform": sf.transforms.NDETransform,
    "base_dist": base_dist,
    "get_net": etorch.models.FFNN,
    "activation": "tanh",
    "inverse_model": True,
    "num_flows": 2,
    "sensitivity": "autograd",
}
EXTRA_M_PARAMS = {
    "neurons": [16],
    "n_hidden_layers": [4],
}

TRAINING_PARAMS = {
    "batch_size": [5000],
    "compute_loss": [sf.nf.get_monte_carlo_elbo_loss()],
    "verbose": True,
}
# extend the previous dict with the zip of this
EXTRA_T_PARAMS = {
    "optimizer": ["ADAM"],
    "num_epochs": [50],
    "learning_rate": [0.01],
    "lr_scheduler": [lr_scheduler],
}

In [ ]:
m_temp_1 = etorch.create_subdictionary_iterator(MODEL_PARAMS, product=True)
m_temp_2 = etorch.create_subdictionary_iterator(EXTRA_M_PARAMS, product=False)
model_params_iter = etorch.add_dictionary_iterators(m_temp_1, m_temp_2, product=True)

t_temp_1 = etorch.create_subdictionary_iterator(TRAINING_PARAMS, product=True)
t_temp_2 = etorch.create_subdictionary_iterator(EXTRA_T_PARAMS, product=False)
training_params_iter = etorch.add_dictionary_iterators(t_temp_1, t_temp_2, product=True)
cv_results = etorch.k_fold_cv_grid(
    model_params=model_params_iter,
    fit=etorch.fit_module,
    training_params=training_params_iter,
    data=data,
    verbose=True,
)

In [ ]:
etorch.plotting.plot_result(
    path_figures=PATH_FIGURES,
    **cv_results,
)

In [ ]:
s = run_skeletons[0]
bonelist = []
for b in s.bones.items():
    for dof in b[1].dof:
        bonelist.append(b[0] + " " + dof)

In [ ]:
i, j = 2, 3
models = cv_results["models"][0][0]
run_point = data_run[:][0]

sample0 = models[0].sample([1000]).detach()
sample1 = models[1].sample([1000]).detach()
df0 = pd.DataFrame({"x": sample0[:, j], "y": sample0[:, i], "Flow": [0] * len(sample0)})
df1 = pd.DataFrame({"x": sample1[:, j], "y": sample1[:, i], "Flow": [1] * len(sample1)})
df = pd.concat((df0, df1), axis=0)
df.index = range(len(df))
sns.displot(df, x="x", y="y", hue="Flow", kind="kde")
plt.scatter(
    run_point[:, j], run_point[:, i], marker="x", color="grey", label="Run samples"
)
walk_points = data_walk[:][0]
plt.scatter(walk_points[:, j], walk_points[:, i], color="green", label="Walk samples")
# plt.xlim(-0.7, 3)
# plt.ylim(-0.8, 0.4)
plt.xlabel(bonelist[choosen[j]])
plt.ylabel(bonelist[choosen[i]])
plt.legend()

plt.savefig(PATH_FIGURES + "/axis_distribution.pdf")
plt.show()

In [ ]:
motion_data = data_walk[:][0]
print(
    "Number in class 1 given  walk data:",
    torch.sum(models[0].log_prob(motion_data) < models[1].log_prob(motion_data)).item(),
    "\nTotal walk data:",
    len(motion_data),
)

motion_data = data_run[:][0]
print(
    "Number in class 1 given  run data:",
    torch.sum(models[0].log_prob(motion_data) > models[1].log_prob(motion_data)).item(),
    "\nTotal run data:",
    len(motion_data),
)

### Model selection

Here we test which dimension that clusters the best
The result of the cell below is the indexes of the chosen dimensions for the model.
**Note**: a bit of manual work might be nesseseary

In [ ]:
for i in nonzero:
    PATH_FIGURES = os.path.join(DIR, SET_NAME + "trial")
    # PATH_FIGURES += f"_{i}"
    if not os.path.exists(PATH_FIGURES):
        os.makedirs(PATH_FIGURES + f"_{i}")
base_dist = dist.Independent(dist.Normal(loc=torch.zeros(1), scale=torch.ones(1)), 1)
MODEL_PARAMS = {
    "model": sf.nf.get_flow,
    "get_transform": sf.transforms.NDETransform,
    "base_dist": base_dist,
    "get_net": etorch.models.FFNN,
    "activation": "tanh",
    "inverse_model": True,
    "num_flows": 2,
    "sensitivity": "autograd",
    "neurons": [16],
    "n_hidden_layers": [4],
}

TRAINING_PARAMS = {
    "batch_size": [5000],
    "compute_loss": [sf.nf.get_monte_carlo_elbo_loss()],
    "optimizer": ["ADAM"],
    "num_epochs": [50],
    "learning_rate": [0.01],
    "lr_scheduler": [lr_scheduler],
}
cv_resultss = []
for joint in tqdm(nonzero):
    # create iterators
    model_params_iter = etorch.create_subdictionary_iterator(MODEL_PARAMS, product=True)
    training_params_iter = etorch.create_subdictionary_iterator(
        TRAINING_PARAMS, product=True
    )
    data = torch.utils.data.TensorDataset(
        wr_angle_tensor_norm[::10, joint : joint + 1],
        priors[::10],
        priors[::10],
    )
    cv_results = etorch.k_fold_cv_grid(
        model_params=model_params_iter,
        fit=etorch.fit_module,
        training_params=training_params_iter,
        data=data,
        verbose=False,
        partial=True,
    )
    etorch.plotting.plot_result(
        path_figures=PATH_FIGURES + f"_{joint}",
        **cv_results,
    )

In [ ]:
dfs = (
    pd.read_csv(PATH_FIGURES + "trial" + f"_{i}/history_plot_0_0.csv")[49::50]
    for i in nonzero
)
loss_df = pd.concat(dfs, axis=0)
loss_df.index = range(47)

In [ ]:
loss_df.sort_values("Validation loss")

In [ ]:
# best cluster per dim,
[1, 0, 14, 33, 26, 1, 20, 32]

In [ ]:
nonzero